In [1]:
import cv2
import numpy as np
import sys
import dlib
import scipy.spatial as sp

In [2]:
#Read points from  text file
def readPoints(path):
    # Create an array of points
    points = []
    # Read points
    with open(path) as file:
        for line in file:
            x,y = line.split()
            points.append((int(x),int(y)))

    return points

In [3]:
# Apply affine tranform calculated using srcTri and sdtTri to src and output an image of size
def applyAffineTransform(src,srcTri,dstTri,size):

    #Given a pair of triangles,find the affine transform.

    warpMat = cv2.getAffineTransform(np.float32(srcTri),np.float32(dstTri))

    #Apply the Affine Transform just foundto the src image
    dst = cv2.warpAffine(src,warpMat,(size[0],size[1]),None,flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REFLECT_101)

    return dst

In [4]:
# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1,img2,img,t1,t2,t,alpha):

    # Find bounding rectangle for each triangle
    # 用一個最小的矩形，把五官的points都包起來
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))

    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []

    for i in range(0,3):
        tRect.append(((t[i][0] - r[0]),(t[i][1]-r[1])))
        t1Rect.append(((t1[i][0]-r1[0]),(t1[i][1]-r1[1])))
        t2Rect.append(((t2[i][0] -r2[0]),(t2[i][1]-r2[1])))

    # Get mask by filling triangles
    mask = np.zeros((r[3],r[2],3),dtype = np.float32)
    cv2.fillConvexPoly(mask,np.int32(tRect),(1.0,1.0,1.0),16,0)

    # Apply warpImage to small rectangular patched
    img1Rect = img1[r1[1]:r1[1]+r1[3],r1[0]:r1[0]+r1[2]]
    img2Rect = img2[r2[1]:r2[1]+r2[3],r2[0]:r2[0]+r2[2]]

    size = (r[2],r[3])
    warpImage1 = applyAffineTransform(img1Rect,t1Rect,tRect,size)
    warpImage2 = applyAffineTransform(img2Rect,t2Rect,tRect,size)

    # Alpha blend rectangular patches
    imgRect = (1.0-alpha) *warpImage1 +alpha*warpImage2

    # Copy triangular region of rectangular patch to tje output image
    print(r[1],r[3],r[0],r[2])
    print(imgRect.shape)
    img[r[1]:r[1]+r[3],r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3],r[0]:r[0]+r[2]]*(1-mask) +imgRect*mask


In [ ]:
if __name__ =='__main__':
    filename1 = "./pure/models/vm.PNG"
    filename2 = "./pure/models/f1.PNG"

    points_txt1 = "./output/img1_68.txt"
    points_txt2  ="./output/img2_68.txt"

    alpha = 0.5

    # Read images
    img1 = cv2.imread(filename1)
    img2 = cv2.imread(filename2)

    # Convertat to float data type
    img1 = np.float32(img1)
    img2 = np.float32(img2)

    # Read array of corresponding points
    points1 = readPoints(points_txt1)
    points2 = readPoints(points_txt2)
    points = []


    # Compute weighted average point coordinate

    for i in range(0,len(points1)):
        x = (1-alpha) *points1[i][0] +alpha *points2[i][0]
        y = (1-alpha)*points1[i][1] + alpha*points2[i][1]
        points.append((x,y))


    imgMorph = np.zeros(img1.shape,dtype = img1.dtype)
    # 計算三角測量索引
    dela = sp.Delaunay
    triang = dela(points)
    for tri in triang.vertices:
        x = tri[0]
        y = tri[1]
        z = tri[2]
        t1 = [points1[x],points1[y],points1[z]]
        t2 = [points2[x],points2[y],points2[z]]
        t = [points[x],points[y],points[z]]
        # Morph one triangle at a time
        morphTriangle(img1,img2,imgMorph,t1,t2,t,alpha)

    # Display Results

    out_img = np.hstack((img1,imgMorph,img2))
    cv2.imwrite("./output/morph_ori_noze_aiki.jpg",imgMorph)
    cv2.imwrite("./output/morph_6.jpg",out_img)
#     cv2.imshow("Morphed Face",out_img)
#     cv2.imshow("Morphed Face",np.uint8(imgMorph))
    cv2.waitKey(0)


739 74 633 78
(74, 78, 3)
754 85 400 69
(85, 69, 3)
754 85 411 122
(85, 122, 3)
686 15 491 107
(15, 107, 3)
680 41 626 30
(41, 30, 3)
682 49 647 32
(49, 32, 3)
682 49 630 26
(49, 26, 3)
720 20 630 18
(20, 18, 3)
698 34 647 65
(34, 65, 3)
730 43 647 65
(43, 65, 3)
730 43 633 78
(43, 78, 3)
748 53 391 61
(53, 61, 3)
748 53 400 69
(53, 69, 3)
908 79 494 39
(79, 39, 3)
772 82 703 8
(82, 8, 3)
812 71 618 89
(71, 89, 3)
853 39 618 86
(39, 86, 3)
838 39 411 122
(39, 122, 3)
800 77 400 22
(77, 22, 3)
894 47 437 71
(47, 71, 3)
876 36 421 87
(36, 87, 3)
838 57 421 112
(57, 112, 3)
894 73 494 37
(73, 37, 3)
894 73 463 45
(73, 45, 3)
733 21 488 19
(21, 19, 3)
753 86 468 65
(86, 65, 3)
686 11 461 137
(11, 137, 3)
680 15 461 166
(15, 166, 3)
696 11 491 81
(11, 81, 3)
706 43 489 31
(43, 31, 3)
696 38 489 31
(38, 31, 3)
720 22 613 21
(22, 21, 3)
700 41 548 46
(41, 46, 3)
736 28 548 46
(28, 46, 3)
700 37 519 53
(37, 53, 3)
736 28 506 44
(28, 44, 3)
706 43 506 43
(43, 43, 3)
740 51 549 45
(51, 45, 3)
74